# Experiment 01: Airline dataset

In this notebook we are going to perform a prediction experiment with XGBoost and LightGBM using the Airline dataset. We are also going to analyze if the dataset has [concept drift](https://en.wikipedia.org/wiki/Concept_drift). 

In this experiment we use [the airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html) to predict arrival delay. The dataset consists of a large amount of records, containing flight arrival and departure details for all the commercial flights within the USA, from October 1987 to April 2008. Its size is around 116 million records and 5.76 GB of memory.

The machine we used was an [Azure NV24 VM](https://azure.microsoft.com/en-gb/blog/azure-n-series-general-availability-on-december-1/), which has [NVIDIA M60](http://www.nvidia.com/object/tesla-m60.html) GPUs. Its operating system is Ubuntu 16.04.

For both XGBoost and LightGBM we compiled from source, to get the last improvements. In XGboost we used the commit `6776292951565c8cd72e69afd9d94de1474f00c0` of May 26th. For LightGBM we used the commit `73968a96829e212b333c88cd44725c8c39c03ad1` of June 2nd. To get these versions and replicate our experiments:
```python
git clone --recursive *url_of_library*
git checkout *oldcommit*
git submodule update --recursive
```

In [4]:
import json
import sys
import warnings
import os
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import pkg_resources
from IPython.display import SVG, display
from bokeh.io import export_svgs
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.loaders import load_airline
from libs.notebook_memory_management import start_watching_memory
from libs.timer import Timer
from libs.utils import get_number_processors
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from sklearn.metrics import (accuracy_score, roc_auc_score, f1_score, precision_score,
                             recall_score)
from libs.metrics import classification_metrics_binary, classification_metrics_binary_prob, binarize_prediction
from toolz import curry
from xgboost import XGBRegressor

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))

warnings.filterwarnings("ignore", category=DeprecationWarning)

System version: 3.5.5 |Anaconda, Inc.| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]
XGBoost version: 0.90
LightGBM version: 2.2.3
In [4] used 0.1250 MiB RAM in 0.11s, total RAM usage 320.88 MiB


The dataset we are going to use in this notebook is huge, therefore we want to monitor the memory consumption. 

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
start_watching_memory()

In [3] used 24.7539 MiB RAM in 10.46s, total RAM usage 320.75 MiB


# 1) XGBoost vs LightGBM benchmark
In the next section we compare both libraries speed, accuracy and other metrics for the dataset of airline arrival delay. 

### Data loading and management

In [5]:
os.environ["MOUNT_POINT"] = "/data1"

In [5] used 0.1602 MiB RAM in 0.10s, total RAM usage 321.04 MiB


In [6]:
%%time
df_plane = load_airline()
print(df_plane.shape)

(115069017, 14)
CPU times: user 1min 33s, sys: 24.7 s, total: 1min 58s
Wall time: 2min 12s
In [6] used 20311.0195 MiB RAM in 133.00s, total RAM usage 20632.05 MiB


In [7]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [7] used 10.9727 MiB RAM in 0.24s, total RAM usage 20643.03 MiB


The first step is to convert the categorical features to numeric features.

In [8]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])
del df_plane

CPU times: user 1min 37s, sys: 18.3 s, total: 1min 55s
Wall time: 2min 6s
In [8] used 6472.5117 MiB RAM in 127.09s, total RAM usage 27115.54 MiB


In [9]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [9] used 0.1953 MiB RAM in 0.12s, total RAM usage 27115.73 MiB


In [10]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')

CPU times: user 57.5 s, sys: 24.5 s, total: 1min 22s
Wall time: 1min 34s
In [10] used -1568.6992 MiB RAM in 94.50s, total RAM usage 25547.04 MiB


In [11]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [11] used 2.6562 MiB RAM in 0.14s, total RAM usage 25549.69 MiB


To simplify the pipeline, we are going to set a classification problem where the goal is to classify wheather a flight has arrived delayed or not. For that we need to binarize the variable `ArrDelay`.

If you want to extend this experiment, you can set a regression problem and try to identify the number of minutes of delay a fight has. Both XGBoost and LightGBM have regression classes.

In [12]:
df_plane_numeric = df_plane_numeric.apply(lambda x: x.astype('int16'))

In [12] used -17783.1523 MiB RAM in 120.96s, total RAM usage 7766.54 MiB


In [13]:
%%time
df_plane_numeric['ArrDelayBinary'] = 1*(df_plane_numeric['ArrDelay'] > 0)

CPU times: user 860 ms, sys: 530 ms, total: 1.39 s
Wall time: 649 ms
In [13] used 25.3477 MiB RAM in 0.78s, total RAM usage 7791.89 MiB


In [14]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


In [14] used 1.5742 MiB RAM in 0.13s, total RAM usage 7793.46 MiB


Once the features are prepared, let's split the dataset into train and test set. We won't use validation for this example (however, you can try to add it).

In [15]:
def split_train_val_test_df(df, val_size=0.2, test_size=0.2):
    train, validate, test = np.split(df.sample(frac=1), 
                                     [int((1-val_size-test_size)*len(df)), int((1-test_size)*len(df))])
    return train, validate, test

In [15] used 0.0000 MiB RAM in 0.10s, total RAM usage 7793.46 MiB


In [16]:
%%time
train, validate, test = split_train_val_test_df(df_plane_numeric, val_size=0, test_size=0.2)
print(train.shape)
print(validate.shape)
print(test.shape)

(92055213, 15)
(0, 15)
(23013804, 15)
CPU times: user 51.6 s, sys: 10.7 s, total: 1min 2s
Wall time: 1min 24s
In [16] used 4141.6719 MiB RAM in 84.19s, total RAM usage 11935.13 MiB


In [17]:
def generate_feables(df):
    X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])]
    y = df['ArrDelayBinary']
    return X,y

In [17] used 0.0586 MiB RAM in 0.10s, total RAM usage 11935.19 MiB


In [18]:
%%time
X_train, y_train = generate_feables(train)
X_val, y_val = generate_feables(validate)
X_test, y_test = generate_feables(test)

CPU times: user 1.01 s, sys: 767 ms, total: 1.78 s
Wall time: 1.78 s
In [18] used 2283.1562 MiB RAM in 1.88s, total RAM usage 14218.35 MiB


In [19]:
del train, validate, test

In [19] used -2458.1016 MiB RAM in 0.24s, total RAM usage 11760.25 MiB


### Training 
Now we are going to create two pipelines, one of XGBoost and one for LightGBM. The technology behind both libraries is different, so it is difficult to compare them in the exact same model setting. XGBoost grows the trees depth-wise and controls model complexity with `max_depth`. Instead, LightGBM uses a leaf-wise algorithm and controls the model complexity by `num_leaves`. As a tradeoff, we use XGBoost with `max_depth=8`, which will have max number leaves of 255, and compare it with LightGBM with `num_leaves=255`. 

In [20]:
results_dict = dict()
num_rounds = 200

In [20] used 0.0000 MiB RAM in 0.10s, total RAM usage 11760.25 MiB


In [21]:
number_processors = get_number_processors()
print(number_processors)

24
In [21] used 0.0000 MiB RAM in 0.11s, total RAM usage 11760.25 MiB


Let's start with the XGBoost classifier.

In [22]:
xgb_clf_pipeline = XGBRegressor(max_depth=8,
                                n_estimators=num_rounds,
                                min_child_weight=30,
                                learning_rate=0.1,
                                scale_pos_weight=2,
                                gamma=0.1,
                                reg_lambda=1,
                                subsample=1,
                                n_jobs=number_processors,
                                random_state=77)

In [22] used 0.0625 MiB RAM in 0.11s, total RAM usage 11760.31 MiB


```python
with Timer() as t:
    xgb_clf_pipeline.fit(X_train, y_train)

results_dict['xgb']={ 'train_time': t.interval }
```

Training XGBoost model with leave-wise growth

In [23]:
xgb_hist_clf_pipeline = XGBRegressor(max_depth=0,
                                    max_leaves=255,
                                    n_estimators=num_rounds,
                                    min_child_weight=30,
                                    learning_rate=0.1,
                                    scale_pos_weight=2,
                                    gamma=0.1,
                                    reg_lambda=1,
                                    subsample=1,
                                    grow_policy='lossguide',
                                    tree_method='hist',
                                    n_jobs=number_processors,
                                    random_state=77)

In [23] used 0.0781 MiB RAM in 0.11s, total RAM usage 11760.39 MiB


In [24]:
with Timer() as t:
    xgb_hist_clf_pipeline.fit(X_train, y_train)

/home/miguel/anaconda/envs/fast/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/miguel/anaconda/envs/fast/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[12:28:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
In [24] used 18801.0820 MiB RAM in 2931.33s, total RAM usage 30561.47 MiB


In [25]:
results_dict['xgb_hist']={
    'train_time': t.interval
}

In [25] used 4.5000 MiB RAM in 0.11s, total RAM usage 30565.97 MiB


Training LightGBM model

In [26]:
lgbm_clf_pipeline = LGBMRegressor(num_leaves=255,
                                  n_estimators=num_rounds,
                                  min_child_weight=30,
                                  learning_rate=0.1,
                                  scale_pos_weight=2,
                                  min_split_gain=0.1,
                                  reg_lambda=1,
                                  subsample=1,
                                  nthread=number_processors,
                                  seed=77)

In [26] used 0.8359 MiB RAM in 0.11s, total RAM usage 30566.80 MiB


In [27]:
with Timer() as t:
    lgbm_clf_pipeline.fit(X_train, y_train)

MemoryError: 

In [27] used -4299.0977 MiB RAM in 33.87s, total RAM usage 26267.71 MiB


In [ ]:
results_dict['lgbm']={
    'train_time': t.interval
}

As it can be seen in the results, given the specific versions and parameters of both XGBoost and LightGBM and in this specific dataset, LightGBM is faster. 

In general terms, leaf-wise algorithms are more efficient, they converge much faster than depth-wise. However, it may cause over-fitting when the data is small or there are too many leaves.

### Evaluation
Now let's evaluate the model in the test set.

```python
with Timer() as t:
    y_prob_xgb = np.clip(xgb_clf_pipeline.predict(X_test), 0.0001, 0.9999)
    
results_dict['xgb']['test_time'] = t.interval
```

In [ ]:
with Timer() as t:
    y_prob_xgb_hist = np.clip(xgb_hist_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [ ]:
results_dict['xgb_hist']['test_time'] = t.interval

In [ ]:
with Timer() as t:
    y_prob_lgbm = np.clip(lgbm_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [ ]:
results_dict['lgbm']['test_time'] = t.interval

### Metrics
We are going to obtain some metrics to evaluate the performance of each of the models.

In [ ]:
#y_pred_xgb = binarize_prediction(y_prob_xgb)
y_pred_xgb_hist = binarize_prediction(y_prob_xgb_hist)
y_pred_lgbm = binarize_prediction(y_prob_lgbm)

```python
report_xgb = classification_metrics_binary(y_test, y_pred_xgb)
report2_xgb = classification_metrics_binary_prob(y_test, y_prob_xgb)
report_xgb.update(report2_xgb)

results_dict['xgb']['performance'] = report_xgb
```

In [ ]:
report_xgb_hist = classification_metrics_binary(y_test, y_pred_xgb_hist)
report2_xgb_hist = classification_metrics_binary_prob(y_test, y_prob_xgb_hist)
report_xgb_hist.update(report2_xgb_hist)

In [ ]:
results_dict['xgb_hist']['performance'] = report_xgb_hist

In [ ]:
report_lgbm = classification_metrics_binary(y_test, y_pred_lgbm)
report2_lgbm = classification_metrics_binary_prob(y_test, y_prob_lgbm)
report_lgbm.update(report2_lgbm)

In [ ]:
results_dict['lgbm']['performance'] = report_lgbm

In [ ]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))

The experiment shows a fairly similar performance in both libraries, being LightGBM slightly better.

In [37]:
del xgb_clf_pipeline, xgb_hist_clf_pipeline, lgbm_clf_pipeline, X_train, X_test, X_val

In [37] used -28481.8008 MiB RAM in 0.18s, total RAM usage 50563.57 MiB


# 2) Concept drift
In this section we are trying to find concept drift in the dataset to check if retraining is valuable.

### Data management
We are going to pack the data yearly to try to find concept drift

In [38]:
initial_year = 1987
num_ini = 5

In [38] used 0.0000 MiB RAM in 0.10s, total RAM usage 50563.57 MiB


In [39]:
def generate_subset_by_year(df, year_ini, year_end):
    return df[df['Year'].isin(range(year_ini, year_end))]

In [39] used 0.0000 MiB RAM in 0.10s, total RAM usage 50563.57 MiB


In [40]:
%%time
subset_base = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + num_ini)
print(subset_base.shape)

(16810190, 15)
CPU times: user 1.54 s, sys: 336 ms, total: 1.87 s
Wall time: 1.85 s
In [40] used 577.1367 MiB RAM in 1.95s, total RAM usage 51140.71 MiB


In [41]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_base.index)]
print(rest_df.shape)

(98258827, 15)
CPU times: user 1min 10s, sys: 4.7 s, total: 1min 14s
Wall time: 1min 14s
In [41] used 10267.1797 MiB RAM in 74.74s, total RAM usage 61407.89 MiB


### Traininig
Let's see what happens when we train on a subset of data and then evaluate in the data of the following years.

In [42]:
X_train, y_train = generate_feables(subset_base)
del(subset_base)

In [42] used -285.5078 MiB RAM in 0.32s, total RAM usage 61122.38 MiB


In [43]:
clf = LGBMClassifier(num_leaves=255,
                    n_estimators=100,
                    min_child_weight=30,
                    learning_rate=0.1,
                    subsample=0.80,
                    colsample_bytree=0.80,
                    seed=42)

In [43] used 0.0000 MiB RAM in 0.10s, total RAM usage 61122.38 MiB


In [44]:
%%time
clf.fit(X_train, y_train)

CPU times: user 28min 52s, sys: 55.9 s, total: 29min 48s
Wall time: 1min 19s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=30, min_split_gain=0, n_estimators=100,
        nthread=-1, num_leaves=255, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=42, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [44] used -2008.7539 MiB RAM in 79.43s, total RAM usage 59113.63 MiB


### Prediction

In [45]:
@curry
def predict_accuracy(clf, test_df):
    X_test, y_test = generate_feables(test_df)
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [45] used 0.0000 MiB RAM in 0.10s, total RAM usage 59113.63 MiB


In [46]:
%%time
accuracy_series = rest_df.groupby('Year').apply(predict_accuracy(clf))
print(accuracy_series)

Year
1992    0.757375
1993    0.756093
1994    0.744296
1995    0.731503
1996    0.722490
1997    0.720751
1998    0.706178
1999    0.700762
2000    0.689513
2001    0.674223
2002    0.680176
2003    0.686131
2004    0.680697
2005    0.673467
2006    0.663597
2007    0.651520
2008    0.636613
dtype: float64
CPU times: user 35min 40s, sys: 23.3 s, total: 36min 4s
Wall time: 2min 9s
In [46] used 5923.4336 MiB RAM in 129.90s, total RAM usage 65037.06 MiB


From the results we can observe that the accuracy of the model gets worse as the years pass on.

### Retraining
Now let's see what happens when we retrain and evaluate in the data of the following years.

In [47]:
new_init = 15

In [47] used 0.0000 MiB RAM in 0.10s, total RAM usage 65037.06 MiB


In [48]:
%%time
subset_retrain = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + new_init) 
print(subset_retrain.shape)

(69425349, 15)
CPU times: user 4.36 s, sys: 1.56 s, total: 5.93 s
Wall time: 5.9 s
In [48] used 2913.2109 MiB RAM in 6.01s, total RAM usage 67950.27 MiB


In [49]:
X_train, y_train = generate_feables(subset_retrain)

In [49] used 1176.3672 MiB RAM in 0.98s, total RAM usage 69126.64 MiB


In [50]:
clf_retrain = LGBMClassifier(num_leaves=255,
                            n_estimators=100,
                            min_child_weight=30,
                            learning_rate=0.1,
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=42)

In [50] used 0.0000 MiB RAM in 0.10s, total RAM usage 69126.64 MiB


In [51]:
%%time
clf_retrain.fit(X_train, y_train)

CPU times: user 1h 40min 56s, sys: 2min 28s, total: 1h 43min 25s
Wall time: 4min 40s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=30, min_split_gain=0, n_estimators=100,
        nthread=-1, num_leaves=255, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=42, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [51] used 4634.6836 MiB RAM in 280.48s, total RAM usage 73761.32 MiB


### Prediction

In [52]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_retrain.index)]
print(rest_df.shape)

(45643668, 15)
CPU times: user 29.3 s, sys: 3.16 s, total: 32.4 s
Wall time: 30.6 s
In [52] used -1183.8203 MiB RAM in 30.74s, total RAM usage 72577.50 MiB


In [53]:
%%time
accuracy_retrain = rest_df.groupby('Year').apply(predict_accuracy(clf_retrain))
print(accuracy_retrain)

Year
2002    0.749985
2003    0.748284
2004    0.724928
2005    0.719791
2006    0.707446
2007    0.695424
2008    0.697949
dtype: float64
CPU times: user 14min 47s, sys: 10.1 s, total: 14min 57s
Wall time: 56 s
In [53] used -1113.7734 MiB RAM in 56.14s, total RAM usage 71463.73 MiB


### Plot

In [54]:
def plot_metrics(metric1, metric2, legend1=None, legend2=None, x_label=None, y_label=None):
    lists = sorted(metric1.items()) 
    x, y = zip(*lists) 
    fig, ax = plt.subplots()
    ax.plot(x, y, label=legend1, color='#5975a4')
    lists2 = sorted(metric2.items()) 
    x2, y2 = zip(*lists2) 
    ax.plot(x2, y2, label=legend2, color='#5f9e6f')
    legend = ax.legend(loc=0)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    plt.show()
    return ax

In [54] used 0.0000 MiB RAM in 0.11s, total RAM usage 71463.73 MiB


In [55]:
from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool

In [55] used 0.0000 MiB RAM in 0.10s, total RAM usage 71463.73 MiB


In [56]:
data_cds = ColumnDataSource(pd.DataFrame({
    'train_acc': accuracy_series,
    'retrain_acc': accuracy_retrain
}))

In [56] used 0.1250 MiB RAM in 0.09s, total RAM usage 71463.86 MiB


In [57]:
# Airline Retrain Results
p = figure(y_axis_label='Accuracy', plot_width=700, plot_height=350, tools="pan,wheel_zoom,box_zoom,reset")
l1 = p.line('Year', 'train_acc', legend=' Train AUC', line_color="#5975a4", source=data_cds, line_width=6, line_cap="round")
p.line('Year', 'retrain_acc', legend=' Retrain AUC', line_color="#a1bae3", source=data_cds, line_width=6, line_cap="round")
l1_hover = HoverTool(renderers=[l1], tooltips=[( 'Train',  '@{train_acc}{0.4f}' ), ( 'Retrain',  '@{retrain_acc}{0.4f}' )], mode='vline')
p.add_tools(l1_hover)
show(p)

In [57] used -27812.4570 MiB RAM in 0.26s, total RAM usage 43651.40 MiB


In [58]:
# SVG: Airline Retrain Results
# Save the plot and display it so that is is visible on Github
p.output_backend = "svg"
export_svgs(p, filename="airline_retrain.svg")
display(SVG('airline_retrain.svg'))

In [58] used 0.1133 MiB RAM in 2.66s, total RAM usage 43651.51 MiB


As it can be seen, the performance is better after retraining. We have found concept drift in this dataset.